In [72]:
import requests
APIKEY = '4IEI37PXMEKHFR2I'
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=60min&apikey={APIKEY}'
r = requests.get(url)
data = r.json()

print(data)

{'Meta Data': {'1. Information': 'Intraday (60min) open, high, low, close prices and volume', '2. Symbol': 'IBM', '3. Last Refreshed': '2021-07-30 20:00:00', '4. Interval': '60min', '5. Output Size': 'Compact', '6. Time Zone': 'US/Eastern'}, 'Time Series (60min)': {'2021-07-30 20:00:00': {'1. open': '140.9800', '2. high': '140.9800', '3. low': '140.9800', '4. close': '140.9800', '5. volume': '527'}, '2021-07-30 18:00:00': {'1. open': '141.1900', '2. high': '141.1900', '3. low': '141.1900', '4. close': '141.1900', '5. volume': '300'}, '2021-07-30 17:00:00': {'1. open': '140.9600', '2. high': '141.0000', '3. low': '140.9600', '4. close': '141.0000', '5. volume': '92357'}, '2021-07-30 16:00:00': {'1. open': '141.0800', '2. high': '141.2600', '3. low': '140.7900', '4. close': '140.9600', '5. volume': '924999'}, '2021-07-30 15:00:00': {'1. open': '141.2200', '2. high': '141.2500', '3. low': '140.9133', '4. close': '141.0700', '5. volume': '247359'}, '2021-07-30 14:00:00': {'1. open': '141.2

In [73]:
import pandas as pd

data_df =pd.DataFrame.from_dict(data['Time Series (60min)'],orient='index').reset_index()
data_df.columns = ['ddate','open','high','low','close','volume']


In [74]:
data_df['ddate'] = data_df['ddate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
for col in data_df.columns[1:]:
    data_df[col] = data_df[col].apply(lambda x: float(x))
data_df['created_time'] = datetime.datetime.now()
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ddate   100 non-null    datetime64[ns]
 1   open    100 non-null    float64       
 2   high    100 non-null    float64       
 3   low     100 non-null    float64       
 4   close   100 non-null    float64       
 5   volume  100 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 4.8 KB


In [84]:
import os
os.listdir()

['.ipynb_checkpoints',
 'alphaVintage_fetch_postgre.py',
 'alphaVintage_postgre.py',
 'testing1.ipynb',
 '__pycache__']

In [76]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ddate         100 non-null    datetime64[ns]
 1   open          100 non-null    float64       
 2   high          100 non-null    float64       
 3   low           100 non-null    float64       
 4   close         100 non-null    float64       
 5   volume        100 non-null    float64       
 6   created_time  100 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(5)
memory usage: 5.6 KB


In [77]:
import psycopg2 as pg
try:
    dbconnect  = pg.connect(
        database = 'airflow',
        user='airflow',password='airflow',
        host='localhost', 
        # Kenapa klo 172.22.0.2 gbsa ya? padahal pgadmin bsa
        port='5432'
    )
except Exception as e:
    print(e)

cursor = dbconnect.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS sandbox.stock_data_IBM (
        ddate timestamp
        , open float
        , high float
        , low float
        , close float
        , volume float
        , created_time timestamp
    )

""")
dbconnect.commit()

In [78]:
from sqlalchemy import create_engine


param_dic = {'user':'airflow','password':'airflow','host':'localhost','database':'airflow'}
connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    param_dic['user'],
    param_dic['password'],
    param_dic['host'],
    param_dic['database']
)
def to_alchemy(df):
    """
    Using a dummy table to test this call library
    """
    engine = create_engine(connect)
    df.to_sql(
        'stock_data_ibm', 
        con=engine, 
        schema='sandbox',
        index=False, 
        if_exists='append'
    )
    print("to_sql() done (sqlalchemy)")

In [79]:
to_alchemy(data_df)

to_sql() done (sqlalchemy)


In [60]:
data_df

,ddate,open,high,low,close,volume
0,2021-07-30 20:00:00,140.980,140.98,140.9800,140.9800,527.0
1,2021-07-30 18:00:00,141.190,141.19,141.1900,141.1900,300.0
2,2021-07-30 17:00:00,140.960,141.00,140.9600,141.0000,92357.0
3,2021-07-30 16:00:00,141.080,141.26,140.7900,140.9600,924999.0
4,2021-07-30 15:00:00,141.220,141.25,140.9133,141.0700,247359.0
...,...,...,...,...,...,...
95,2021-07-21 14:00:00,141.110,141.29,140.9200,141.0800,404212.0
96,2021-07-21 13:00:00,141.045,141.28,140.9300,141.1100,333896.0
97,2021-07-21 12:00:00,141.135,141.29,140.5700,141.0450,530997.0
98,2021-07-21 11:00:00,141.010,141.33,140.5000,141.1350,716346.0
